In [1]:
import pandas as pd

import numpy as np 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import  DistilBertTokenizer
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments


c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv("D:\Datasets\movie-genre-prediction\\train.csv")
test = pd.read_csv("D:\Datasets\movie-genre-prediction\\test.csv")

In [3]:
train.head()

,Unnamed: 0,id,movie_name,synopsis,genre
0,0,44978,Super Me,A young scriptwriter starts bringing valuable ...,fantasy
1,1,50185,Entity Project,A director and her friends renting a haunted h...,horror
2,2,34131,Behavioral Family Therapy for Serious Psychiat...,This is an educational video for families and ...,family
3,3,78522,Blood Glacier,Scientists working in the Austrian Alps discov...,scifi
4,4,2206,Apat na anino,Buy Day - Four Men Widely - Apart in Life - By...,action


In [4]:
train['encoded_categories'] = train['genre'].astype('category').cat.codes

In [5]:
data_text = train['synopsis'].to_list()
data_labels = train['encoded_categories'].to_list()

In [6]:
x_train,x_valid,y_train,y_valid = train_test_split(data_text,data_labels,test_size=0.2, random_state=0)

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [8]:
train_encoding = tokenizer(x_train,truncation=True,padding=True)
val_encoding = tokenizer(x_valid,truncation=True,padding=True)

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encoding),
    y_train,
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encoding),
    y_valid
))

Fine Tunning with TFTrainer Class

In [10]:
training_args = TFTrainingArguments(
    output_dir="E:\output_tf",
    num_train_epochs = 7,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    warmup_steps=500,
    logging_dir="E:\log_tf",
    eval_steps=100)

In [11]:
# with training_args.strategy.scope():
trainer_model = TFBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 10)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['distilbert.transformer.layer.5.sa_layer_norm.bias', 'distilbert.transformer.layer.1.attention.out_lin.weight', 'distilbert.transformer.layer.5.attention.out_lin.bias', 'distilbert.transformer.layer.4.attention.q_lin.bias', 'distilbert.transformer.layer.2.ffn.lin2.bias', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.5.attention.k_lin.bias', 'distilbert.embeddings.word_embeddings.weight', 'distilbert.transformer.layer.2.attention.q_lin.bias', 'distilbert.transformer.layer.3.attention.out_lin.weight', 'distilbert.transformer.layer.4.ffn.lin1.bias', 'distilbert.transformer.layer.4.ffn.lin2.bias', 'distilbert.transformer.layer.0.a

In [12]:
trainer = TFTrainer(
    model=trainer_model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset
)

c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\trainer_tf.py:118: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key fr

In [ ]:
# trainer_model.compile(optimizer='adam',loss='SparseCategoricalCrossentropy',metrics=['accuracy'])
# trainer_model.fit(train_encoding,y_train,epochs=3)

ValueError: Failed to find data adapter that can handle input: <class 'transformers.tokenization_utils_base.BatchEncoding'>, (<class 'list'> containing values of types {"<class 'int'>"})

In [ ]:
trainer.train()

Saving and Loading of model

In [ ]:
save_directory = "E:\saved_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


NameError: name 'model' is not defined

Loading the pretrained Model

In [ ]:
tokenizer_fine_tuned = DistilBertTokenizer